In [81]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [82]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

Allgemein:

Lines vielleicht global?
Oder zumindest als Parameter?

# Wiedergeben einer bereits gespielten Partie

In diesem Bereich werden Funktionen definiert, die das Wiedergeben einer Partie ermöglichen. 
Hierfür wird eine Spiel-Historie eingelesen. 
Historien werden bei der Ausführung des ``03_play_against_ai.ipynb`` Notebooks erstellt und in dem Ordner ``Played_Games`` hinterlegt. 
Am Ende dieses Notebook wird der Spielverlauf mit einem selbst festgelegten Tempo der Züge dargestellt.

## Importieren des Spielverlaufs

Als ersten Schritt muss eine Schach-Historie eingelesen werden. 
Hierfür wird die Funktion ``get_lines_from_file()`` definiert, die für einen mitgegebenen relativen Pfad zu einer Schach-Historie (``file_name``) die Zeilen der Datei ohne Zeilenumbruch (``"\n"``) zurückgibt.
Die Datei, welche unter `file_name` zu finden ist, muss dem Format entsprechen, dass in ``03_play_against_ai`` gespeichert wird.  
Dieses speichert in der ersten Zeile die Fen der Spielsituation. In den folgenden Zeilen werden die Züge der Spieler (jeweils zwei Halbzüge) gespeichert.

> TODO: Dateiformat beschreiben?
> TODO: Methode beschreiben - ich verstehe nicht was hier passiert.

In [83]:
def get_lines_from_file():
    move_file = open("Played_Games/" + g_filename + ".txt", "r")
    all_lines = move_file.readlines()

    # Remove '\n' from the lines
    lines = [all_lines[x][:-1] for x in range(len(all_lines)) if len(all_lines[x]) > 12]
    if len(all_lines) == len(lines) + 1:
        lines += [all_lines[-1]]
    return lines

## Auslesen der Startposition und Züge

Um den zuvor gespielten Ablauf wiedergeben zu können, wird ein `board` Objekt der `chess` Bibliothek benötigt.
Dieses wird im Verlauf des Notebooks mit Figuren gefüllt, und modifiziert.

In [84]:
g_board = chess.Board(None)

Mit der Funktion ``set_board`` wird das zuvor erstellte Board initialisiert.
Für die Initialisierung wird die als Parameter (`fen`) übergebene Fen, welche in der ersten Zeile der History-Datei gespeichert ist verwendet.

In [85]:
def setup_board(fen):
    global g_board
    g_board.set_fen(fen)

Um den Spielverlauf darstellen zu können, werden die Spielzüge benötigt. Diese können mit der Funktion ``get_moves_from_file`` ausgewertet werden. 
Es wird über die Züge, welche in der History-Datei gespeichert waren (`fullmove_history`) iteriert. Diese werden in Halbzüge aufgeteilt, zu `python-chess` Objekten konvertiert, und in einer Liste `moves` gespeichert.
Die Liste der Halbzüge wird zurückgegeben.

In [86]:
def halfmoves_from_history(fullmove_history):
    moves = []
    for line in fullmove_history:
        split_moves = line.split(' ')
        moves = [chess.Move.from_uci(m) for m in split_moves[1:] if len(m)]
        moves.extend(moves)
    return moves

## Spiel wiederholen

Die Wiederholung des Spiels kann nun mit der Ausführung der Funktion ``start_replay`` begonnen werden. Diese benötigt als Parameter den relativen Pfad zu einer Spielhistorie (``file_name``) und zusätzlich eine Anzahl an Sekunden, die nach der Ausführung eines Zuges gewartet werden sollen (``delay``). Das Spiel wird daraufhin in der Konsole für den Nutzer schrittweise angezeigt.

In [87]:
def replay_moves(movelist):
    global g_board
    for move in movelist:
        time.sleep(g_delay)
        clear_output()
        g_board.push(move)
        display(g_board)

In [88]:
def start_replay():
    # setup
    history_data = get_lines_from_file()
    fen = history_data[0]
    halfmoves = halfmoves_from_history(history_data[1:]) 
    print(halfmoves)
    
    # run
    setup_board(fen)
    display(g_board)
    replay_moves(halfmoves)
    print("finished!")
    

## Einstellungen
Der Nutzer muss vor Beginn einer Ausgabe Einstellungen über die folgenden globalen Variablen tätigen:
* `g_filename`: Dateiname der abzuspielenden Datei (muss im Verzeichnis `Played_Games` liegen), ohne Dateiendung .txt
* `g_delay`: Verzögerung in Sekunden zwischen den Zügen

In [89]:
g_filename = "Move-History_2022-05-07 11_41_19"
g_delay = 1

In [90]:
start_replay()


AssertionError: push() expects move to be pseudo-legal, but got a6g6 in 8/4k3/6Q1/8/8/8/8/4K3